[Набор](https://archive.ics.uci.edu/dataset/2/adult) данных UCI Adult (качать не надо, все есть в репозитории): классификация людей с помощью демографических данных для прогнозирования, зарабатывает ли человек более \$ 50 000 в год.

Описание признаков:

**Age** – возраст, количественный признак  
**Workclass** – тип работодателя, количественный признак  
**fnlwgt** – итоговый вес обьекта, количественный признак    
**Education** – уровень образования, качественный признак  
**Education_Num** – количество лет обучения, количественный признак  
**Martial_Status** – семейное положение, категориальный признак  
**Occupation** – профессия, категориальный признак  
**Relationship** – тип семейных отношений, категориальный признак    
**Race** – раса, категориальный признак  
**Sex** – пол, качественный признак  
**Capital_Gain** – прирост капитала, количественный признак  
**Capital_Loss** – потери капитала, количественный признак  
**Hours_per_week** – количество часов работы в неделю, количественный признак  
**Country** – страна, категориальный признак  
   
Целевая переменная: **Target** – уровень заработка, категориальный (бинарный) признак

1. Выполнить разведочный анализ данных, заполнить пропуски, преобразовать категориальные данные в числовые:
    - Кодируем категориальные признаки 'Workclass', 'Education', 'Martial_Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country'. Это можно сделать с помощью метода pandas get_dummies.
    
2. В тестовой выборке не оказалось Голландии. Завести необходимый признак из нулей.

3. Обучить LogisticRegression(). Использовать в качестве baseline.
4. Обучить k-NN, SVC, Naive Bayes, DecisionTree, RandomForest
5. Обучить SVC с такими параметрами {'classifier__gamma':[0.001, 0.01, 0.1, 1, 10, 100], 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]}
6. Обучить DecisionTreeClassifier c параметрами tree_params = {'max_depth': range(2,11)}
7. Обучить RandomForestClassifier c параметрами forest_params = {'max_depth': range(10, 21), 'max_features': range(5, 105, 10)}
8. Использовать кроссвалидацию и GridSearchCV.
9. Сравнить все модели с baseline
10. Вывести confusion matrix


In [6]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, HalvingGridSearchCV, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [7]:
# Загрузка данных
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
column_names = ["Age", "Workclass", "fnlwgt", "Education", "Education_Num", "Martial_Status", "Occupation", "Relationship", "Race", "Sex", "Capital_Gain", "Capital_Loss", "Hours_per_week", "Country", "Target"]
data = pd.read_csv(url, names=column_names, sep=',\s*', engine='python')

In [8]:
# Шаг 1: Предобработка данных
# Кодирование категориальных признаков
categorical_features = ["Workclass", "Education", "Martial_Status", "Occupation", "Relationship", "Race", "Sex", "Country"]
data_encoded = pd.get_dummies(data, columns=categorical_features)
data_encoded

,Age,fnlwgt,Education_Num,Capital_Gain,Capital_Loss,Hours_per_week,Target,Workclass_?,Workclass_Federal-gov,Workclass_Local-gov,...,Country_Portugal,Country_Puerto-Rico,Country_Scotland,Country_South,Country_Taiwan,Country_Thailand,Country_Trinadad&Tobago,Country_United-States,Country_Vietnam,Country_Yugoslavia
0,39,77516,13,2174,0,40,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,<=50K,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,257302,12,0,0,38,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
32557,40,154374,9,0,0,40,>50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
32558,58,151910,9,0,0,40,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False
32559,22,201490,9,0,0,20,<=50K,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [9]:
# Шаг 2: Завести признак для Голландии (предположим, он называется 'From_Holland')
data_encoded["From_Holland"] = 0

In [10]:
# Шаг 3: Обучение Logistic Regression (бейзлайн)
X = data_encoded.drop("Target", axis=1)
y = data_encoded["Target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)
logistic_accuracy = logistic_model.score(X_test, y_test)
logistic_accuracy2 = logistic_model.score(X_train, y_train)
print("Logistic Regression Accuracy Test:", logistic_accuracy)
print("Logistic Regression Accuracy Train:", logistic_accuracy2)

Logistic Regression Accuracy Test: 0.7996315062183326
Logistic Regression Accuracy Train: 0.7971437346437347


In [11]:
# Шаг 4: Обучение других моделей
# k-NN
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)
knn_accuracy = knn_model.score(X_test, y_test)
print("k-NN Accuracy:", knn_accuracy)

# SVC
svc_model = SVC()
svc_model.fit(X_train, y_train)
svc_accuracy = svc_model.score(X_test, y_test)
print("SVC Accuracy:", svc_accuracy)

# Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
nb_accuracy = nb_model.score(X_test, y_test)
print("Naive Bayes Accuracy:", nb_accuracy)

# Decision Tree
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)
dt_accuracy = dt_model.score(X_test, y_test)
print("Decision Tree Accuracy:", dt_accuracy)

# Random Forest
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_accuracy = rf_model.score(X_test, y_test)
print("Random Forest Accuracy:", rf_accuracy)

k-NN Accuracy: 0.7785966528481498
SVC Accuracy: 0.7956394902502687
Naive Bayes Accuracy: 0.7991708889912482
Decision Tree Accuracy: 0.8166743436204514
Random Forest Accuracy: 0.8593582066635959


In [13]:
# Шаг 5: Обучение SVC с подбором параметров
grid_search_svc = HalvingGridSearchCV(SVC(), param_grid={ 'gamma': [0.001, 0.01, 0.1, 1, 10, 100],'C': [0.001, 0.01, 0.1, 1, 10, 100]}, n_jobs=-1)
grid_search_svc.fit(X_train, y_train)
best_svc_model = grid_search_svc.best_estimator_
best_svc_accuracy = best_svc_model.score(X_test, y_test)
print("Best SVC Accuracy:", best_svc_accuracy)

Best SVC Accuracy: 0.75894365115922


In [14]:
# Шаг 6: Обучение DecisionTreeClassifier с подбором параметров
tree_params = {'max_depth': range(2, 11)}
grid_search_tree = GridSearchCV(DecisionTreeClassifier(), tree_params, cv=5)
grid_search_tree.fit(X_train, y_train)
best_tree_model = grid_search_tree.best_estimator_
best_tree_accuracy = best_tree_model.score(X_test, y_test)
print("Best Decision Tree Accuracy:", best_tree_accuracy)

Best Decision Tree Accuracy: 0.8615077537233226


In [15]:
# Шаг 7: Обучение RandomForestClassifier с подбором параметров
forest_params = {'max_depth': range(10, 21), 'max_features': range(5, 105, 10)}
grid_search_forest = GridSearchCV(RandomForestClassifier(), forest_params, cv=5)
grid_search_forest.fit(X_train, y_train)
best_forest_model = grid_search_forest.best_estimator_
best_forest_accuracy = best_forest_model.score(X_test, y_test)
print("Best Random Forest Accuracy:", best_forest_accuracy)

Best Random Forest Accuracy: 0.8693382465837556


In [16]:
# Шаг 8: Кросс-валидация и GridSearchCV уже выполнены в предыдущих шагах

# Шаг 9: Сравнение с бейзлайном
print("Baseline Logistic Regression Accuracy:", logistic_accuracy)
print("k-NN Accuracy:", knn_accuracy)
print("SVC Accuracy:", svc_accuracy)
print("Naive Bayes Accuracy:", nb_accuracy)
print("Decision Tree Accuracy:", dt_accuracy)
print("Random Forest Accuracy:", rf_accuracy)
print("Best SVC Accuracy:", best_svc_accuracy)
print("Best Decision Tree Accuracy:", best_tree_accuracy)
print("Best Random Forest Accuracy:", best_forest_accuracy)

Baseline Logistic Regression Accuracy: 0.7996315062183326
k-NN Accuracy: 0.7785966528481498
SVC Accuracy: 0.7956394902502687
Naive Bayes Accuracy: 0.7991708889912482
Decision Tree Accuracy: 0.8166743436204514
Random Forest Accuracy: 0.8593582066635959
Best SVC Accuracy: 0.75894365115922
Best Decision Tree Accuracy: 0.8615077537233226
Best Random Forest Accuracy: 0.8693382465837556


In [17]:
# Шаг 10: Confusion Matrix
y_pred = best_forest_model.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion)

Confusion Matrix:
 [[4698  244]
 [ 607  964]]


# ДЗ №2 <a name='dz2'></a>

1. Разбить датасет digits на обучающую и тестовую выборку, 85% и 15% соответственно.
2. Найти лучший классификатор (и лучшие параметры) для этого набора данных.
3. Вывести матрицу несоответствия

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits

# Загружаем датасет
digits = load_digits()

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.15, random_state=42)


In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Определяем параметры для поиска
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

# Создаем классификатор
svm = SVC()

# Инициируем Grid Search
grid_search = GridSearchCV(svm, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Назодим лучшие параметры
best_params = grid_search.best_params_
best_classifier = grid_search.best_estimator_


In [7]:
from sklearn.metrics import confusion_matrix

y_pred = best_classifier.predict(X_test)
confusion = confusion_matrix(y_test, y_pred)

print("Матрица несоответствия:")
print(confusion)


Матрица несоответствия:
[[23  0  0  0  0  0  0  0  0  0]
 [ 0 18  0  0  0  0  0  0  0  0]
 [ 0  0 26  0  0  0  0  0  0  0]
 [ 0  0  0 23  0  1  0  0  0  0]
 [ 0  0  0  0 37  0  0  0  0  0]
 [ 0  0  0  0  0 33  0  0  0  0]
 [ 0  0  0  0  0  0 30  0  0  0]
 [ 0  0  0  0  0  0  0 25  0  1]
 [ 0  0  0  0  0  0  0  0 19  0]
 [ 0  0  0  0  0  0  0  1  0 33]]
